In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [4]:
# Load API Credentials
with open('/Users/dmusl/.secret/tmdb_api.json') as f:
    login = json.load(f)

In [5]:
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [6]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [7]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [8]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/oMsxZEvz9a708d49b6UdZK1KAo5.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 76.048,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [9]:
info['budget']

63000000

In [10]:
info['revenue']

463517383

In [11]:
info['imdb_id']

'tt0133093'

Searching with IMDB_ID

Try searching by the IMDb number. For example, What was the budget of Tom and Jerry which had an IMDb id of "tt1361336"?

In [14]:
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']

50000000

In [15]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

# Defining Our Function
The function should accept the movie_id as an argument.
Make sure to replace the current test id we used with a movie_id variable in your function definition!
It should return a dictionary of results that includes certification.

In [ ]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # save the .info .releases dictionaries
    info = movie.info()

    releases = movie.releases()
    #Loop through countries in releases
    for c in releases ['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] =='US':
            ## save a "certification" key in info with the certification
            info['certification'] = c['certification']
            
    return info

In [ ]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

### Handling this Error with Try and Except
We can handle this error by using a try and except statement. Note: remember that errors are also called "Exceptions".

In brief, a try-except statement works like an if-else statement:

Just like an if statement, we start with the word try and a : followed by a new line indented inside of the try statement.
The code that runs into errors is the code that we want to try to run.
​If there are no errors, the code will continue running, as usual.
Unlike an if statement, a try statement MUST have an except statement too.
The except block includes the code to run if the try statement hits an error. 

In [ ]:
## testing our function by looping through a list of ids
import pandas as pd
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
​
​# Loop through test_ids
results = []
for movie_id in test_ids:

      try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except: 
        pass
    
pd.DataFrame(results)

### Saving Our Error Messages

Additionally, we can save the error message in a variable, which we will call e, just by saying except Exception as e: instead of just saying except:.

Once we've saved the error message, we can display the error if we want to see the error message if we want to know to see the details for every error.  Since it is just being displayed, it won't actually stop the loop from running, but we will still see the information.

In [ ]:
# Using a try-except statement to catch and display an error message
​for movie_id in test_ids:   
​     try:
​        movie_info = get_movie_with_rating(movie_id)
​        results.append(movie_info)
​    
​    # To see the error message when it happens:
​    except Exception as e:
​         display(e)

However, If we are looping through a large collection of movies like you will be doing for the project, this could get very cluttered and will interfere with our progress bars.

Instead, we can create a list where we can append the movie id and error message. This way, we can check this list later to see how many movies caused an error - and which ones.

In [ ]:
# Using a try-except statement to catch and append the error message to a list
import pandas as pd
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

In [ ]:
# Now we can check our error list to see if we have any errors and what they were.
print(f"- Number of errors: {len(errors)}")
errors

# Efficient TMDB API Calls

In [ ]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)